# Tree detection inference

In [ ]:
# Import required libraries
from pathlib import Path
import torch

from tree_detection_framework.constants import DATA_FOLDER
from tree_detection_framework.detection.detector import DeepForestDetector
from tree_detection_framework.detection.models import DeepForestModule
from tree_detection_framework.postprocessing.postprocessing import multi_region_NMS
from tree_detection_framework.preprocessing.preprocessing import (
    create_dataloader,
    visualize_dataloader,
)

## Define constants
The behavior of this notebook can be changed by adjusting these values.

In [ ]:
# The path to a folder of raster images, e.g. orthomosaics stor
RASTER_FOLDER_PATH = Path(DATA_FOLDER, "emerald-point-ortho")
RASTER_FILE_PATH = Path(RASTER_FOLDER_PATH, "ortho.tif")
# Prediction file path
OUTPUT_PREDICTIONS_FILE_PATH = Path(
    DATA_FOLDER, "emerald-point-ortho-deep-forest-predictions.gpkg"
)
# The size of the chips in pixels
CHIP_SIZE = 1024
# The stride between chips in pixels
CHIP_STRIDE = 768
# The spatial resolution that the data is sampled to in meters/pix
OUTPUT_RESOLUTION = 0.05
# The number of samples in a batch
BATCH_SIZE = 4
# Any detections below this threshold will be discarded
MIN_CONFIDENCE_THRESHOLD = 0.3
# Detections that overlap more than this threshold will be suppressed
IOU_NMS_THESHOLD = 0.3

# How many tiles to show
N_VIS_TILES = 3

# Setup the parameters dictionary
MODEL_PARAMS = {
    "backbone": "retinanet",
    "num_classes": 1,
}
# The device to use for inference, GPU or CPU
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Create a dataloader to generate samples

In [ ]:
dataloader = create_dataloader(
    raster_folder_path=RASTER_FOLDER_PATH,
    chip_size=CHIP_SIZE,
    chip_stride=CHIP_STRIDE,
    output_resolution=OUTPUT_RESOLUTION,
    batch_size=BATCH_SIZE,
)

Show example samples

In [ ]:
visualize_dataloader(dataloader, n_tiles=N_VIS_TILES)

Create a DeepForest detector model

In [ ]:
# This model uses pretrained weights from DeepForest
df_module = DeepForestModule(MODEL_PARAMS).to(DEVICE)
df_detector = DeepForestDetector(df_module)

Generate predictions for each tile in the dataloader

In [ ]:
outputs = df_detector.predict(dataloader)

Plot the predictions for all overlapping tiles colored by the prediction confidence

In [ ]:
outputs.plot(visualization_column="score", bounds_color="r", raster_file=RASTER_FILE_PATH)

Run nonmax suppression and across all the tiles and remove low-confidence detections

In [ ]:
NMS_outputs = multi_region_NMS(
    outputs,
    confidence_column="score",
    iou_theshold=IOU_NMS_THESHOLD,
    min_confidence=MIN_CONFIDENCE_THRESHOLD,
)

Show the result again after running NMS

In [ ]:
NMS_outputs.plot(
    visualization_column="score", bounds_color="r", raster_file=RASTER_FILE_PATH
)

Save out the predictions to a geospatial file

In [ ]:
NMS_outputs.save(OUTPUT_PREDICTIONS_FILE_PATH)